In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas requests bs4

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import urllib.request
import time

In [ ]:
def get_source_file(url):
    page = urllib.request.urlopen(url)
    return BeautifulSoup(page.read(), "html.parser")

#Kiểm tra việc get code và bóc tách bằng bs4

In [ ]:
url = "https://www.anphatpc.com.vn/laptop-dell-vostro-3400-70253900-core-i5-1135g7-8gb-256gb-intel-iris-xe-14.0-inch-fhd-win-10-office_id39197.html"
soup = get_source_file(url)
price = soup.findAll('div', class_="popup-spec")[0].findAll('tr')
data = []
for i in price:
    if len(i.findAll("td")) == 2:
        data.append(i.findAll("td"))
result = []

for i in data:
    for x in i:
      try:
          result.append(x.findAll("span")[0].text.replace("\xa0", ""))
      except:
          print(x)

PRODUCER = []
CPU = []
RAM = []
DISK = []
DISPLAY = []
GPU = []
PRODUCER.append(result[result.index("Hãng sản xuất") + 1])
CPU.append(result[result.index("Bộ vi xử lý") + 1])
RAM.append(result[result.index("Dung lượng") + 1])
DISK.append(result[result.index("Tốc độ vòng quay") - 1])
DISPLAY.append(result[result.index("Màn hình") + 1])
if result[result.index("Card màn hình") + 1] != "Wireless":
    GPU.append(result[result.index("Card màn hình") + 1])
else:
    GPU.append("")

<td style="width: 578.2pt; padding: .75pt .75pt .75pt .75pt;" width="771">
<p style="margin: 0cm; margin-bottom: .0001pt; text-align: center;"> </p>
</td>
Laptop Dell
Intel® Core™ i5-1135G7
8GB(8GBx1) DDR4 3200Mhz
256GB M.2 PCIe NVMe SSD
14.0-inch FHD (1920 x 1080) An ti-glare LED Backlight Non-Touch Narrow Border WVA Display
Intel Iris Xe Graphics


#Đọc dữ liệu vào (tên laptop : link sản phẩm)

In [2]:
%cd "/content/drive/MyDrive/KHDL"

/content/drive/MyDrive/KHDL


In [ ]:
data = pd.read_csv("Crawl_Search_Phong_Vu.csv", sep="\t")
print(data)
product_name = [x for x in data["Name"]]
product_url = [x for x in data["Link"]]

In [ ]:
product = dict(zip(product_name, product_url))

In [ ]:
product

# Crawl chi tiết của An Phát

In [ ]:
def processing_price(price):
    if price.strip().upper() == "LIÊN HỆ":
        return "Liên hệ"          
    price = price.strip().replace(".", "").replace(" đ", "")
    return int(price)

In [ ]:
def crawl_info(url):
    soup = get_source_file(url)
    x = 0
    # get price of item
    try:
        OLD_PRICE = processing_price(soup.findAll('del', class_="font-500")[0].text)
        NEW_PRICE = processing_price(soup.findAll('b', class_='text-18 js-pro-total-price')[0].text)
    except:
        NEW_PRICE = processing_price(soup.findAll('b', class_='text-18 js-pro-total-price')[0].text)
        OLD_PRICE = new_price
    # get hardware configuration
    pre_data = soup.findAll('div', class_="popup-spec")[0].findAll('tr')
    data = []
    # remove empty data
    for element in pre_data:
        if len(element.findAll("td")) == 2:
            data.append(element.findAll("td"))
    result = []
    for element in data:
        for ele in element:
            try:
                result.append(ele.findAll("span")[0].text.replace("\xa0", ""))
            except:
                x += 1
    try:
        PRODUCER = result[result.index("Hãng sản xuất") + 1]
    except:
        PRODUCER = ""
    try:
        CPU = result[result.index("Bộ vi xử lý") + 1]
        RAM = result[result.index("Dung lượng") + 1]
        DISK = result[result.index("Tốc độ vòng quay") - 1]
        DISPLAY = result[result.index("Màn hình") + 1]
        if result[result.index("Card màn hình") + 1] != "Wireless":
            GPU = result[result.index("Card màn hình") + 1]
        else:
            GPU = ""
    except:
        return "None", "None", "None", "None", "None", "None", "None", "None"

    return PRODUCER, CPU, RAM, GPU, DISK, DISPLAY, OLD_PRICE, NEW_PRICE 


In [ ]:
NAME = []
PRODUCER = []
CPU = []
RAM = []
GPU = []
DISK = []
DISPLAY = []
OLD_PRICE = []
NEW_PRICE = []
print("Total: ", len(product.items()))
index = 1
for product_name, product_url in product.items():
    print("Crawl: ", index)
    if product_name.split(" ")[0] != "Laptop":
        continue
    time.sleep(3)
    try:
        producer, cpu, ram, gpu, disk, display, old_price, new_price = crawl_info(product_url)
        if NAME != "None" and PRODUCER != "None" and CPU != "None" and RAM != "None" and GPU != "None" and DISK != "None" and DISPLAY != "None" and OLD_PRICE != "None" and NEW_PRICE != "None":
            NAME.append(product_name)
            PRODUCER.append(producer)
            CPU.append(cpu)
            RAM.append(ram)
            GPU.append(gpu)
            DISK.append(disk)
            DISPLAY.append(display)
            OLD_PRICE.append(old_price)
            NEW_PRICE.append(new_price)
    except:
        print('Processing index: ', index)
        time.sleep(5)
        if NAME != "None" and PRODUCER != "None" and CPU != "None" and RAM != "None" and GPU != "None" and DISK != "None" and DISPLAY != "None" and OLD_PRICE != "None" and NEW_PRICE != "None":
            NAME.append(product_name)
            PRODUCER.append(producer)
            CPU.append(cpu)
            RAM.append(ram)
            GPU.append(gpu)
            DISK.append(disk)
            DISPLAY.append(display)
            OLD_PRICE.append(old_price)
            NEW_PRICE.append(new_price)
    index += 1

data = pd.DataFrame({"Name":NAME, "Producer":PRODUCER, "CPU":CPU, "RAM":RAM, "GPU":GPU, "DISK":DISK, "DISPLAY":DISPLAY, "Old Price": OLD_PRICE, "New Price":NEW_PRICE})
data.to_csv("Product_An_Phat.csv", sep="\t", index=None)

In [ ]:
df = pd.read_csv("Product_An_Phat.csv", sep="\t")
df

#Crawl chi tiết Phong Vũ

In [ ]:
def get_API(url):
    product_id = url.split('sku=')[-1]
    api = 'https://phongvu.vn/api/product/'
    return api + product_id

In [ ]:
PRODUCER = []
CPU = []
RAM = []
GPU = []
DISK = []
DISPLAY = []
OLD_PRICE = []
NEW_PRICE = []
NAME = []

In [ ]:
import requests
import json
def crawl_product(product):
    index = 1
    for name, url in product.items():
        print('Crawl index: ', index)
        NAME.append(name)
        raw_data = requests.get(get_API(url)).text
        product_data = json.loads(raw_data)
        temp = product_data['result']['product']['productDetail']['attributeGroups']
        for i in range(0, len(temp)):
            if temp[i]['name'] == 'Thương hiệu':
                PRODUCER.append(temp[i]['value'])
            elif temp[i]['name'] == 'CPU':
                CPU.append(temp[i]['value'])
            elif temp[i]['name'] == 'RAM':
                RAM.append(temp[i]['value'])
            elif temp[i]['name'] == 'Chip đồ họa':
                GPU.append(temp[i]['value'])
            elif temp[i]['name'] == 'Lưu trữ':
                DISK.append(temp[i]['value'])
            elif temp[i]['name'] == 'Màn hình':
                DISPLAY.append(temp[i]['value'])
        temp = product_data['result']['product']['prices'][0]
        NEW_PRICE.append(temp['latestPrice'])
        OLD_PRICE.append(temp['terminalPrice'])
        index += 1
    data = pd.DataFrame({"Name":NAME, "Producer":PRODUCER, "CPU":CPU, "RAM":RAM, "GPU":GPU, "DISK":DISK, "DISPLAY":DISPLAY, "Old Price": OLD_PRICE, "New Price":NEW_PRICE})
    data.to_csv("Product_Phong_Vu.csv", sep="\t", index=None)


In [ ]:
crawl_product(product)

In [ ]:
df = pd.read_csv("Product_Phong_Vu.csv", sep="\t")
df

,Name,Producer,CPU,RAM,GPU,DISK,DISPLAY,Old Price,New Price
0,Laptop Acer Nitro 5 AN515-52-51LW (NH.Q3LSV.00...,ACER,Intel Core i5-8300H ( 2.3 GHz - 4.0 GHz / 8MB...,1 x 8GB DDR4 2666MHz ( 2 Khe cắm / Hỗ trợ t...,NVIDIA GeForce GTX 1050Ti 4GB GDDR5 / Intel...,128GB SSD M.2 SATA / 1TB HDD 5400RPM,"15.6"" ( 1920 x 1080 ) Full HD IPS không cả...",24040000,24040000
1,Laptop Acer Swift 3 SF313-51-56UW (NX.H3ZSV.00...,ACER,Intel Core i5-8250U ( 1.6 GHz - 3.4 GHz / 6MB...,8GB Onboard DDR4 Không nâng cấp được ),Intel UHD Graphics 620,256GB SSD M.2 NVMe /,"13.3"" ( 1920 x 1080 ) Full HD IPS không cả...",26490000,26490000
2,"Laptop MSI GP65 Leopard 9SD-224VN (15"" FHD/i7-...",MSI,Intel Core i7-9750H ( 2.6 GHz - 4.5 GHz / 12M...,2 x 8GB DDR4 2666MHz ( 2 Khe cắm / Hỗ trợ t...,NVIDIA GeForce GTX 1660Ti 6GB GDDR6 / Intel...,512GB SSD M.2 NVMe /,"15.6"" ( 1920 x 1080 ) Full HD IPS 144Hz , k...",37990000,35990000
3,Laptop Acer Aspire 3 A315-56-37DV (NX.HS5SV.00...,ACER,Intel Core i3-1005G1 ( 1.2 GHz - 3.4 GHz / 4M...,1 x 4GB Onboard DDR4 2400MHz ( 1 Khe cắm / ...,Intel UHD Graphics,256GB SSD M.2 NVMe /,"15.6"" ( 1920 x 1080 ) Full HD không cảm ứn...",12990000,12190000
4,Laptop MICROSOFT Surface Pro 7 QWT-00001 ( 12....,MICROSOFT,Intel Core i3-1005G1 ( 1.2 GHz - 3.4 GHz / 4M...,4GB LPDDR4,Intel UHD Graphics,128GB SSD /,"12.3"" ( 2736 x 1824 ) cảm ứng , HD webcam",23990000,23290000
5,"Laptop MSI GF63 Thin 9SCSR 829VN ( 15.6"" Full ...",MSI,Intel Core i5-9300H ( 2.4 GHz - 4.1 GHz / 8MB...,1 x 8GB DDR4 2666MHz ( 2 Khe cắm / Hỗ trợ t...,NVIDIA GeForce GTX 1650Ti 4GB GDDR6 / Intel...,512GB SSD M.2 NVMe /,"15.6"" ( 1920 x 1080 ) Full HD IPS 144Hz , k...",22990000,20190000
6,"Laptop HP 340s G7 240Q4PA ( 14"" Full HD/Intel ...",HP,Intel Core i3-1005G1 ( 1.2 GHz - 3.4 GHz / 4M...,1 x 4GB DDR4 2666MHz ( 2 Khe cắm / Hỗ trợ t...,Intel UHD Graphics,256GB SSD M.2 NVMe /,"14"" ( 1920 x 1080 ) Full HD IPS không cảm ...",12990000,12990000
7,"Laptop Dell Vostro 14 3405 V4R53500U001W ( 14""...",Dell,AMD Ryzen 5 3500U ( 2.1 GHz - 3.7 GHz / 4MB /...,1 x 4GB DDR4 2400MHz ( 2 Khe cắm / Hỗ trợ t...,AMD Radeon Vega 8 Graphics,256GB SSD M.2 NVMe /,"14"" ( 1920 x 1080 ) Full HD WVA không cảm ...",16490000,16190000
8,Laptop ACER Swift 3 SF314-510G-57MR NX.A10SV.0...,ACER,Intel Core i5-1135G7 ( 4.2 GHz / 8MB / 4 nhân...,8GB Onboard LPDDR4X 2666MHz Không nâng cấp...,Intel Iris Xe Max Graphics,512GB SSD M.2 NVMe /,"14"" ( 1920 x 1080 ) Full HD IPS không cảm ...",23990000,23990000
9,"Laptop Dell Vostro 14 3405 V4R53500U003W ( 14""...",Dell,AMD Ryzen 5 3500U ( 2.1 GHz - 3.7 GHz / 4MB /...,1 x 8GB DDR4 2400MHz ( 2 Khe cắm / Hỗ trợ t...,AMD Radeon Vega 8 Graphics,512GB SSD M.2 NVMe /,"14"" ( 1920 x 1080 ) Full HD WVA không cảm ...",18490000,18090000


In [8]:
data = pd.read_csv("Crawl_Search_HN_CPT.csv", sep="\t")
print(data)
product_name = [x for x in data["Name"]]
product_url = [x for x in data["Link"]]

                                                  Name                                               Link
0    LAPTOP ASUS ZENBOOK UX325EA-KG363T (I5 1135G7/...  https://www.hanoicomputer.vn/laptop-asus-zenbo...
1    LAPTOP ASUS GAMING ROG FLOW 13 GV301QC-K6029T ...  https://www.hanoicomputer.vn/laptop-asus-gamin...
2    LAPTOP LENOVO THINKPAD X1 GEN 9 (20XW0076VN) (...  https://www.hanoicomputer.vn/laptop-lenovo-thi...
3    LAPTOP LENOVO LEGION 7 16ACHG6 (82N60039VN) (R...  https://www.hanoicomputer.vn/laptop-lenovo-leg...
4    LAPTOP ACER GAMING PREDATOR TRITON 300 (PT315-...  https://www.hanoicomputer.vn/laptop-acer-gamin...
..                                                 ...                                                ...
916  LAPTOP LENOVO LEGION 5-15ACH6A (82NW003BVN) (R...  https://www.hanoicomputer.vn/laptop-lenovo-leg...
917  LAPTOP LENOVO LEGION 5-15ACH6A (82NW003CVN) (R...  https://www.hanoicomputer.vn/laptop-lenovo-leg...
918  LAPTOP DELL INSPIRON 3501 (70253897) (I5 